# Poultry Disease Classification from Fecal Images

This notebook classifies poultry diseases based on fecal images into four classes:

- **Healthy**
- **Coccidiosis (cocci)**
- **Salmonella (salmo)**
- **Newcastle Disease (ncd)**

> ⚙️ *The model is optimized for Mac M1 with 8GB RAM.* (Farhan Mashrur)
- Model initially developed with Ahmed Abdulla (Teammate) for Mac M2 


### GPU availability Testing :

In [2]:
import tensorflow as tf
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices('GPU'))
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


/Users/farhanmashrur/Desktop/cds/avian_alert/tf-metal-env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


TensorFlow version: 2.16.2
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs Available: 1


 ## 1. Importing Libraries


In [3]:
import os
import time
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers

# Enable mixed precision for Apple Silicon (M1/M2)
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print('Modules loaded')

Modules loaded


## 2. Enable GPU Acceleration (for Mac M1)
#### For Mac M1, we are using TensorFlow-MacOS and Metal plugin are installed.

In [4]:
try:
    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print("GPU acceleration enabled on M1")
    else:
        print("No GPU found")
except Exception as e:
    print("GPU acceleration not available:", e)


# 3. Image Settings for Mac M1 (8GB RAM)
IMG_SIZE = (160, 160)  # Reduced for efficiency
CHANNELS = 3
BATCH_SIZE = 32        # Lower to 16 if memory issues occur
IMG_SHAPE = (IMG_SIZE[0], IMG_SIZE[1], CHANNELS)

GPU acceleration enabled on M1


## 3) Part 1 : Callback Setup and Initialization


In [5]:
class MyCallback(keras.callbacks.Callback):
    def __init__(self, model, patience=1, stop_patience=3, threshold=0.9, factor=0.5, batches=None, epochs=None, ask_epoch=None):
        super(MyCallback, self).__init__()
        self._model = model
        self.patience = patience 
        self.stop_patience = stop_patience
        self.threshold = threshold
        self.factor = factor
        self.batches = batches
        self.epochs = epochs
        
        self.count = 0
        self.stop_count = 0
        self.best_epoch = 1
        
        try:
            self.current_lr = 0.001  # Default fallback
            if hasattr(model.optimizer, 'learning_rate'):
                lr = model.optimizer.learning_rate
                if hasattr(lr, 'numpy'):
                    self.current_lr = float(lr.numpy())
            elif hasattr(model.optimizer, 'lr'):
                lr = model.optimizer.lr
                if hasattr(lr, 'numpy'):
                    self.current_lr = float(lr.numpy())
        except:
            pass
            
        self.initial_lr = self.current_lr
        self.highest_tracc = 0.0
        self.lowest_vloss = np.inf
        self.best_weights = self._model.get_weights()
        self.initial_weights = self._model.get_weights()
